##1. 설치

In [ ]:
!pip install fastapi uvicorn nest-asyncio flask pyngrok flask_ngrok

In [ ]:
!ngrok config add-authtoken 30s47Zrpt0z8JIXJRCLMMklctLS_43GWGBsYeAL5k558po9Xo

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


https://dashboard.ngrok.com/agents

## 2. flask

In [ ]:
template = """
<!DOCTYPE html>
<html>
<head>
    <title>주가 예측</title>
</head>
<body>
    <h2>종가 예측 (t-1, t-2, t-3 → t)</h2>

    <h3>1. 모델 학습</h3>
    <form action="/train" method="post">
        티커: <input type="text" name="ticker" value="AAPL"><br>
        기간: <input type="text" name="period" value="6mo"><br>
        <button type="submit">학습하기</button>
    </form>

    <h3>2. 예측하기</h3>
    <form action="/predict" method="post">
        t-1: <input type="text" name="t1"><br>
        t-2: <input type="text" name="t2"><br>
        t-3: <input type="text" name="t3"><br>
        <button type="submit">예측</button>
    </form>
</body>
</html>
"""

In [ ]:
import nest_asyncio
import yfinance as yf
import pickle
import threading
from flask import Flask, request, render_template_string
from pyngrok import ngrok
from sklearn.ensemble import RandomForestRegressor

# 중복 실행 방지
nest_asyncio.apply()

# Flask 앱 정의
app = Flask(__name__)

@app.route('/')
def home():
    return render_template_string(template)

# ngrok으로 포트 5000 노출
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")

def run_flask():
    app.run(port=5000)

# 쓰레드로 실행
threading.Thread(target=run_flask).start()


Public URL: NgrokTunnel: "https://f6aa3d83b5d0.ngrok.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'


##3.routing 추가

In [ ]:
# 분류 또는 회귀를 수행
# yfinance, pykrx에서 데이터를 받거나, 파일을 읽으셔서 학습
# 학습된 모형 피클 파일로..
# 예측을 지원
# train, predict, + a
# 런타임 재시작, ngrok 대시보드 -> agent-> stop 이후 다시 적용
# apikey 개인키를 사용해주세요.
# 첫화면 form을 이용해주세요.


In [ ]:
template = """
<!DOCTYPE html>
<html>
<head>
    <title>주가 예측</title>
</head>
<body>
    <h2>종가 예측 (t-1, t-2, t-3 → t)</h2>

    <h3>1. 모델 학습</h3>
    <form action="/train" method="post">
        티커: <input type="text" name="ticker" value="AAPL"><br>
        기간: <input type="text" name="period" value="6mo"><br>
        <button type="submit">학습하기</button>
    </form>

    <h3>2. 예측하기</h3>
    <form action="/predict" method="post">
        t-1: <input type="text" name="t1"><br>
        t-2: <input type="text" name="t2"><br>
        t-3: <input type="text" name="t3"><br>
        <button type="submit">예측</button>
    </form>
</body>
</html>
"""

In [ ]:
import nest_asyncio
import yfinance as yf
import pickle
import threading
from flask import Flask, request, render_template_string
from pyngrok import ngrok
from sklearn.ensemble import RandomForestRegressor

# 중복 실행 방지
nest_asyncio.apply()

# Flask 앱 정의
app = Flask(__name__)

@app.route('/')
def home():
    return render_template_string(template)

@app.route('/train', methods=['POST'])
def train():
    ticker = request.form.get('ticker', 'AAPL')
    period = request.form.get('period', '6mo')

    df = yf.download(ticker, period=period)
    df = df[["Close"]]
    df["t-1"] = df["Close"].shift(1)
    df["t-2"] = df["Close"].shift(2)
    df["t-3"] = df["Close"].shift(3)
    df["target"] = df["Close"]
    df.dropna(inplace=True)

    X = df[["t-1", "t-2", "t-3"]]
    y = df["target"]

    model = RandomForestRegressor()
    model.fit(X, y)
    with open("model.pkl", "wb") as f:
        pickle.dump(model, f)

    return f"{ticker} 종목에 대해 {period} 기간으로 {len(X)}건 학습 완료"

@app.route('/predict', methods=['POST'])
def predict():
    try:
        t1 = float(request.form.get('t1', 0))
        t2 = float(request.form.get('t2', 0))
        t3 = float(request.form.get('t3', 0))
        with open("model.pkl", "rb") as f:
            model = pickle.load(f)
        prediction = model.predict([[t1,t2,t3]])
        return f"예측된 종가: {prediction}"
    except ValueError as e:
        return f"입력 오류: {e}", 400

# ngrok으로 포트 5000 노출
public_url = ngrok.connect(5001)
print(f"Public URL: {public_url}")

def run_flask():
    app.run(port=5001)

# 쓰레드로 실행
threading.Thread(target=run_flask).start()


Public URL: NgrokTunnel: "https://22d037054768.ngrok.app" -> "http://localhost:5001"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5001
INFO:werkzeug:Press CTRL+C to quit


In [1]:
template = """
<!DOCTYPE html>
<html>
<head>
    <title>주가 예측</title>
</head>
<body>
    <h2>종가 예측 (t-1, t-2, t-3 → t)</h2>

    <h3>1. 모델 학습</h3>
    <form action="/train" method="post">
        티커: <input type="text" name="ticker" value="AAPL"><br>
        기간: <input type="text" name="period" value="6mo"><br>
        <button type="submit">학습하기</button>
    </form>

    <h3>2. 예측하기</h3>
    <form action="/predict" method="post">
        t-1: <input type="text" name="t1"><br>
        t-2: <input type="text" name="t2"><br>
        t-3: <input type="text" name="t3"><br>
        <button type="submit">예측</button>
    </form>

    <h3>3. DB저장</h3>
    <form action="/db" method="post">
        t-1: <input type="text" name="t1"><br>
        t-2: <input type="text" name="t2"><br>
        t-3: <input type="text" name="t3"><br>
        <button type="submit">저장</button>
    </form>



</body>
</html>
"""

In [2]:
import sqlite3

# 1. 데이터베이스 연결
conn = sqlite3.connect("fin.db")  # 메모리 DB는 ":memory:"
cursor = conn.cursor()

# 2. 테이블 생성
cursor.execute('''
CREATE TABLE IF NOT EXISTS fin (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    t1 INTEGER,
    t2 INTEGER,
    t3 INTEGER,
    pred INTEGER
)
''')



In [3]:
import nest_asyncio
import yfinance as yf
import pickle
import threading
from flask import Flask, request, render_template_string
from pyngrok import ngrok
from sklearn.ensemble import RandomForestRegressor

# 중복 실행 방지
nest_asyncio.apply()

# Flask 앱 정의
app = Flask(__name__)

@app.route('/')
def home():
    return render_template_string(template)

@app.route('/train', methods=['POST'])
def train():
    ticker = request.form.get('ticker', 'AAPL')
    period = request.form.get('period', '6mo')

    df = yf.download(ticker, period=period)
    df = df[["Close"]]
    df["t-1"] = df["Close"].shift(1)
    df["t-2"] = df["Close"].shift(2)
    df["t-3"] = df["Close"].shift(3)
    df["target"] = df["Close"]
    df.dropna(inplace=True)

    X = df[["t-1", "t-2", "t-3"]]
    y = df["target"]

    model = RandomForestRegressor()
    model.fit(X, y)
    with open("model.pkl", "wb") as f:
        pickle.dump(model, f)

    return f"{ticker} 종목에 대해 {period} 기간으로 {len(X)}건 학습 완료"

@app.route('/predict', methods=['POST'])
def predict():
    try:
        t1 = float(request.form.get('t1', 0))
        t2 = float(request.form.get('t2', 0))
        t3 = float(request.form.get('t3', 0))
        with open("model.pkl", "rb") as f:
            model = pickle.load(f)
        prediction = model.predict([[t1,t2,t3]])
        return f"예측된 종가: {prediction}"
    except ValueError as e:
        return f"입력 오류: {e}", 400

@app.route('/db', methods=['POST'])
def db():
    try:
        t1 = float(request.form.get('t1', 0))
        t2 = float(request.form.get('t2', 0))
        t3 = float(request.form.get('t3', 0))
        with open("model.pkl", "rb") as f:
            model = pickle.load(f)
        prediction = model.predict([[t1,t2,t3]])

        # 데이터 삽입
        conn = sqlite3.connect("fin.db")  # 메모리 DB는 ":memory:"
        cursor = conn.cursor()
        cursor.execute("INSERT INTO fin (t1, t2, t3, pred) VALUES (?, ?, ?, ?)", (t1,t2,t3,prediction))
        conn.commit()  # 반드시 커밋해야 저장됨

        return f"DB저장 완료"
    except ValueError as e:
        return f"입력 오류: {e}", 400


# ngrok으로 포트 5000 노출
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")

def run_flask():
    app.run(port=5000)

# 쓰레드로 실행
threading.Thread(target=run_flask).start()


# 7. 종료
conn.close()



Public URL: NgrokTunnel: "https://36947097a926.ngrok.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


In [4]:
        # 데이터 삽입
        conn = sqlite3.connect("fin.db")  # 메모리 DB는 ":memory:"
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM fin")
        rows = cursor.fetchall()
        for row in rows:
            print(row)

(1, 1, 2, 3, 4)
(2, 1, 2, 3, 4)
(3, 1, 2, 3, 4)
(4, 1, 2, 3, 4)
(5, 1, 2, 3, b'\x14\xaeG\t\xd8\xd4g@')
(6, 1, 2, 3, b'=\n\xd7[9\xe6g@')
(7, 100, 150, 200, b'\xf6(\\\x7f\xb8\xf5b@')
